In [20]:
text = """['call', 'sell', 'believ', 'maersk', 'financi', 'perform', 'vulner', 'lower', 'freight', 'rate', 'downward', 'trend', 'due', 'continu', 'subdu', 'demand', 'eas', 'port', 'congest', 'releas', 'capac', 'back', 'furthermor', 'think', 'neg', 'feedback', 'loop', 'freight', 'rate', 'continu', 'think', 'custom', 'may', 'seek', 'renegoti', 'given', 'price', 'differenti', 'contract', 'spot', 'also', 'overhang', 'situat', 'may', 'amplifi', 'entri', 'new', 'vessel', 'project', 'increas', 'industri', 'suppli', 'nonetheless', 'posit', 'term', 'strategi', 'grow', 'logist', 'busi', 'complement', 'core', 'busi', 'upsid', 'risk', 'includ', 'strong', 'econom', 'growth', 'improv', 'ship', 'activ', 'posit', 'revers', 'freight', 'rate', 'effect', 'cost', 'target', 'price', 'maersk', 'reflect', 'broadli', 'line', 'peer', 'averag']"""

----------
    Chunk-size Approach

In [2]:
from transformers import BertForSequenceClassification, BertTokenizer
import torch

c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [21]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")

In [22]:
def get_sentiment(tokens):
    outputs = model(**tokens)   # pass tokens to get output
    probabilities = torch.nn.functional.softmax(outputs[0], dim=-1) # Convert retrieved outputs to probabilities
    return probabilities

In [23]:
tokens = tokenizer.encode_plus(text, add_special_tokens=False)
tokens

Token indices sequence length is longer than the specified maximum sequence length for this model (1758 > 512). Running this sequence through the model will result in indexing errors


{'input_ids': [50118, 134, 4, 289, 5969, 21770, 46058, 104, 3243, 248, 19739, 3063, 19713, 2371, 3808, 6034, 2808, 4307, 41603, 6390, 8640, 1889, 5758, 2606, 25815, 2118, 31299, 7111, 8662, 15231, 18827, 221, 7949, 2604, 30018, 9162, 3858, 7981, 25049, 1723, 5089, 26747, 6597, 2571, 18347, 4, 50118, 50118, 3762, 11, 292, 1791, 266, 519, 2281, 11196, 6, 8, 4583, 3608, 21416, 1446, 16, 55, 1537, 566, 3240, 3362, 6, 604, 6, 8, 6689, 12568, 4, 152, 16, 1341, 430, 31, 97, 613, 9571, 13588, 30, 5, 12612, 6078, 6, 1664, 1908, 1581, 4, 44, 48, 1185, 17, 27, 241, 164, 7, 33, 55, 4478, 867, 1174, 85, 17, 27, 29, 9861, 15, 201, 7, 11427, 6, 7, 6296, 6, 7, 9263, 2476, 963, 4, 17, 46, 50118, 50118, 14009, 13513, 34798, 271, 6, 10, 320, 3484, 8, 3080, 4589, 8, 122, 1031, 736, 9, 5, 5366, 6944, 2534, 824, 13, 2108, 5123, 6, 3307, 1130, 9588, 1503, 7, 434, 11, 2304, 21416, 35, 44, 48, 1106, 47, 300, 65, 11, 292, 8195, 14, 33, 34326, 19, 21416, 1174, 646, 23742, 9, 1148, 742, 32, 164, 7, 386, 1576, 24,

In [24]:
input_ids = tokens['input_ids']
attention_mask = tokens['attention_mask']

In [25]:
len(tokens['input_ids'])

1758

In [26]:
input_ids[:10]

[50118, 134, 4, 289, 5969, 21770, 46058, 104, 3243, 248]

In [27]:
type(tokens['input_ids'])

list

In [28]:
start = 0
window_length = 512

total_len = len(input_ids)

loop = True

while loop:
    end = start + window_length
    if end >= total_len:
        loop = False
        end = total_len
    print(f"start = {start}")
    print(f"end = {end}")
    start = end

start = 0
end = 512
start = 512
end = 1024
start = 1024
end = 1536
start = 1536
end = 1758


In [29]:
total_len = len(input_ids)

In [30]:
def chunk_text_to_window_size_and_predict_proba(input_ids, attention_mask, total_len):

    proba_list = []
    start = 0
    window_length = 510
    loop = True

    while loop:
        end = start + window_length

        if end >= total_len:
            loop = False
            end = total_len
            
        # Define the text chunk
        input_ids_chunk = input_ids[start:end]
        attention_mask_chunk = attention_mask[start:end]

        # Append [CLS] and [SEP]
        input_ids_chunk = [101] + input_ids_chunk + [102]
        attention_mask_chunk = [1] + attention_mask_chunk + [1]

        # Convert to PyTorch tensors
        input_dict = {
            'input_ids': torch.Tensor([input_ids_chunk]).long(),
            'attention_mask': torch.Tensor([attention_mask_chunk]).int()
        }

        outputs = model(**input_dict)
        probabilities = torch.nn.functional.softmax(outputs[0], dim=-1)
        proba_list.append(probabilities)

        start = end  # Update start for the next chunk

    return proba_list

In [31]:
proba_list = chunk_text_to_window_size_and_predict_proba(input_ids, attention_mask, total_len)

In [32]:
proba_list

[tensor([[6.4013e-04, 3.0657e-01, 6.9279e-01]], grad_fn=<SoftmaxBackward0>),
 tensor([[0.0471, 0.9515, 0.0014]], grad_fn=<SoftmaxBackward0>),
 tensor([[0.3914, 0.6037, 0.0050]], grad_fn=<SoftmaxBackward0>),
 tensor([[0.0006, 0.4391, 0.5603]], grad_fn=<SoftmaxBackward0>)]

In [33]:
stacks = torch.stack(proba_list)
stacks

tensor([[[6.4013e-04, 3.0657e-01, 6.9279e-01]],

        [[4.7112e-02, 9.5149e-01, 1.4015e-03]],

        [[3.9138e-01, 6.0366e-01, 4.9529e-03]],

        [[6.1176e-04, 4.3910e-01, 5.6029e-01]]], grad_fn=<StackBackward0>)

In [34]:
shape = stacks.shape
shape

torch.Size([4, 1, 3])

In [35]:
torch.reshape(stacks, (shape[0], shape[2] ) )

tensor([[6.4013e-04, 3.0657e-01, 6.9279e-01],
        [4.7112e-02, 9.5149e-01, 1.4015e-03],
        [3.9138e-01, 6.0366e-01, 4.9529e-03],
        [6.1176e-04, 4.3910e-01, 5.6029e-01]], grad_fn=<ViewBackward0>)

In [36]:
def get_mean_from_proba(proba_list):
    """
    This function computes the mean probabilities of class predictions over all the chunks.

    Args:
        proba_list (List[torch.Tensor]): List of probability tensors for each chunk.

    Returns:
        mean (torch.Tensor): Mean of the probabilities across all chunks.
    """
    
    # Ensures that gradients are not computed, saving memory
    with torch.no_grad():
        # Stack the list of tensors into a single tensor
        stacks = torch.stack(proba_list)

        # Resize the tensor to match the dimensions needed for mean computation
        stacks = stacks.resize(stacks.shape[0], stacks.shape[2])

        # Compute the mean along the zeroth dimension (i.e., the chunk dimension)
        mean = stacks.mean(dim = 0)
        
    return mean

mean = get_mean_from_proba(proba_list)
# tensor([0.0767, 0.1188, 0.8045])

c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\_tensor.py:868: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


In [37]:
torch.argmax(mean).item()

1

----------

In [ ]:
csv_path = r'C:\Users\hp\Desktop\lars\project\module 5\final_scores.csv'

df.to_csv(csv_path, index=False)

print('Final Scores file saved to Module 5')